# 5003 TITLE

* 可以用schema初始化数据类型，但是还需要对应位置，就神烦，不整了，GTMD
* 如果用这个方法，需要在读取数据的时候增加参数schema，这里设置的结构就叫schema，所以直接写schema=schema
* 还需要导入包：`from pyspark.sql.types import *`
* schema写法：`schema = StructType([StructField("id", IntegerType()),StructField("title", StringType()),StructField("tagline", StringType()),StructField("vote_average", FloatType()),StructField("vote_count", IntegerType())])`
* 当你读取csv格式错误的时候，你就加一个`multiLine=True,escape='"'`，专治各种不服

## UDF是解决DataFrame问题的通法😂

In [1]:
# 导入各种用到的包
# 返回值类型支持
from pyspark.sql.types import ArrayType
from pyspark.sql.types import MapType
from pyspark.sql.types import StringType
from pyspark.sql.types import NullType
from pyspark.sql.types import DoubleType
# UDF模块
from pyspark.sql.functions import udf

* 形如`[{'id': 16,...},{????},]`
* 这次处理的结构大致是list里面含有maps，但是整体的类型又是string。

In [2]:
# 类型转换：
typetransfer = udf(lambda x:eval(x), ArrayType(MapType(StringType(),StringType())))

In [3]:
# 把genres提取出来，再建立一个UDF来实现这个功能。
takegenres = udf(lambda x: [i['name'] for i in x], StringType())

In [4]:
# 去掉外面的中括号，这是上面的代码处理list时候的衍生物
rmbracket = udf(lambda x:','.join(x), StringType())

In [5]:
# 取出导演
getdirector = udf(lambda x:[i['name'] for i in x if i['job'] == 'Director'],StringType())
# 取出主演
getcharacter = udf(lambda x:[i['name'] for i in x],StringType())

In [6]:
# 取出关键词
getkws = udf(lambda x:[i['name'] for i in x],StringType())

In [7]:
# 切割词语,返回string
cutwords_str = udf(lambda x:str(x).split(' '),StringType())
# 返回Array,专门针对tagline
cutwords = udf(lambda x:str(x).split(' '),ArrayType(StringType()))

In [8]:
# 这里都是没有用到的UDF，不用跑这部分代码
# 只保留两个主演，第一版，总是报奇怪的错
get2character = udf(lambda x:[eval(x)[0],eval(x)[1]],StringType())
# 如果有至少一个就返回第一个演员，可以跑出来
get1character = udf(lambda x:[x[0] if len(x)>=1 else []],StringType())
# 如果有至少两个就返回前两个演员，不可以跑出来，注意下面的代码虽然会被执行，但是立刻就被覆盖了
# 想测试的话记得改一下
get2character = udf(lambda x:[x[0],x[1] if len(x)>=2 else []],StringType())
# 改写成这样跑出来了,而且直接给了不带中括号的
get2character = udf(lambda x:','.join([x[0],x[1]]) if len(x)>=2 else '',StringType())
# 思考一下，似乎你不用在乎演员出现的次数
# 如果出现的很少的角色演员，会被过滤，如果是很大牌的演员，只要出现就有影响力
# 放弃这个部分，因为毁掉了3000+个数据

# 处理movies_metadata.csv

In [9]:
df1 = spark.read.csv('/Users/apple/Desktop/5003-project/moviedata/TMD/movies_metadata.csv',multiLine=True,escape='"', header=True, inferSchema=True)

In [10]:
# 一眼就能看出来这个具体结构一眼看不出来，那就不看了吧
df1.show()

+-----+---------------------+--------+--------------------+--------------------+-----+---------+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+------------+---------+-------+--------------------+--------+--------------------+--------------------+-----+------------+----------+
|adult|belongs_to_collection|  budget|              genres|            homepage|   id|  imdb_id|original_language|      original_title|            overview|popularity|         poster_path|production_companies|production_countries|release_date|  revenue|runtime|    spoken_languages|  status|             tagline|               title|video|vote_average|vote_count|
+-----+---------------------+--------+--------------------+--------------------+-----+---------+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+------------+---------+-------+-----------

In [11]:
# 看一看这个数据的结构和类型，以及包含哪些特征
print(df1)
print('------------------分界线------------------')
df1.printSchema()
# 有的数据类型并不如你所愿，记得转换

DataFrame[adult: string, belongs_to_collection: string, budget: string, genres: string, homepage: string, id: string, imdb_id: string, original_language: string, original_title: string, overview: string, popularity: string, poster_path: string, production_companies: string, production_countries: string, release_date: string, revenue: bigint, runtime: double, spoken_languages: string, status: string, tagline: string, title: string, video: boolean, vote_average: double, vote_count: int]
------------------分界线------------------
root
 |-- adult: string (nullable = true)
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- id: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- post

In [12]:
# 抽出我们需要的那些特征,让他们看上去更简洁
bmd = df1.select('genres','id','tagline','title','vote_average','vote_count')
bmd

DataFrame[genres: string, id: string, tagline: string, title: string, vote_average: double, vote_count: int]

In [13]:
# 把id的数据类型转换成int，并放在前面
bmd2 = bmd.select(bmd['id'].astype('int'),'genres','tagline','title','vote_average','vote_count')
bmd2

DataFrame[id: int, genres: string, tagline: string, title: string, vote_average: double, vote_count: int]

In [14]:
# 这里bmd2中genres一列的内容很乱，需要单独处理，取出关键词。
# 使用前面提到的UDF函数来解决这个问题。
bmd2.show()

+-----+--------------------+--------------------+--------------------+------------+----------+
|   id|              genres|             tagline|               title|vote_average|vote_count|
+-----+--------------------+--------------------+--------------------+------------+----------+
|  862|[{'id': 16, 'name...|                null|           Toy Story|         7.7|      5415|
| 8844|[{'id': 12, 'name...|Roll the dice and...|             Jumanji|         6.9|      2413|
|15602|[{'id': 10749, 'n...|Still Yelling. St...|    Grumpier Old Men|         6.5|        92|
|31357|[{'id': 35, 'name...|Friends are the p...|   Waiting to Exhale|         6.1|        34|
|11862|[{'id': 35, 'name...|Just When His Wor...|Father of the Bri...|         5.7|       173|
|  949|[{'id': 28, 'name...|A Los Angeles Cri...|                Heat|         7.7|      1886|
|11860|[{'id': 35, 'name...|You are cordially...|             Sabrina|         6.2|       141|
|45325|[{'id': 28, 'name...|The Original Bad ...| 

In [15]:
# 先转换成类型，方便之后的操作
bmd3 = bmd2.select('id',typetransfer('genres').alias('genres'),'tagline','title','vote_average','vote_count')
# 取出关键词
bmd4 = bmd3.select('id',takegenres('genres').alias('genres'),'tagline','title','vote_average','vote_count')
bmd5 = bmd4.select('id',rmbracket('genres').alias('genres'),'tagline','title','vote_average','vote_count')
bmd5.show()

+-----+--------------------+--------------------+--------------------+------------+----------+
|   id|              genres|             tagline|               title|vote_average|vote_count|
+-----+--------------------+--------------------+--------------------+------------+----------+
|  862|Animation,Comedy,...|                null|           Toy Story|         7.7|      5415|
| 8844|Adventure,Fantasy...|Roll the dice and...|             Jumanji|         6.9|      2413|
|15602|      Romance,Comedy|Still Yelling. St...|    Grumpier Old Men|         6.5|        92|
|31357|Comedy,Drama,Romance|Friends are the p...|   Waiting to Exhale|         6.1|        34|
|11862|              Comedy|Just When His Wor...|Father of the Bri...|         5.7|       173|
|  949|Action,Crime,Dram...|A Los Angeles Cri...|                Heat|         7.7|      1886|
|11860|      Comedy,Romance|You are cordially...|             Sabrina|         6.2|       141|
|45325|Action,Adventure,...|The Original Bad ...| 

In [16]:
# 再单独取出一些数据
bmd6 = bmd5.select('id','genres','vote_average','vote_count')
bmd6.show()

+-----+--------------------+------------+----------+
|   id|              genres|vote_average|vote_count|
+-----+--------------------+------------+----------+
|  862|Animation,Comedy,...|         7.7|      5415|
| 8844|Adventure,Fantasy...|         6.9|      2413|
|15602|      Romance,Comedy|         6.5|        92|
|31357|Comedy,Drama,Romance|         6.1|        34|
|11862|              Comedy|         5.7|       173|
|  949|Action,Crime,Dram...|         7.7|      1886|
|11860|      Comedy,Romance|         6.2|       141|
|45325|Action,Adventure,...|         5.4|        45|
| 9091|Action,Adventure,...|         5.5|       174|
|  710|Adventure,Action,...|         6.6|      1194|
| 9087|Comedy,Drama,Romance|         6.5|       199|
|12110|       Comedy,Horror|         5.7|       210|
|21032|Family,Animation,...|         7.1|       423|
|10858|       History,Drama|         7.1|        72|
| 1408|    Action,Adventure|         5.7|       137|
|  524|         Drama,Crime|         7.8|     

# 处理cresits.csv

In [17]:
cres = spark.read.csv('/Users/apple/Desktop/5003-project/moviedata/TMD/credits.csv',multiLine=True,escape='"', header=True, inferSchema=True)

In [18]:
# 和处理上一个数据集中genres一栏的方法可以说是一样的
# 取出导演和主演
cres.show()
cres

+--------------------+--------------------+-----+
|                cast|                crew|   id|
+--------------------+--------------------+-----+
|[{'cast_id': 14, ...|[{'credit_id': '5...|  862|
|[{'cast_id': 1, '...|[{'credit_id': '5...| 8844|
|[{'cast_id': 2, '...|[{'credit_id': '5...|15602|
|[{'cast_id': 1, '...|[{'credit_id': '5...|31357|
|[{'cast_id': 1, '...|[{'credit_id': '5...|11862|
|[{'cast_id': 25, ...|[{'credit_id': '5...|  949|
|[{'cast_id': 1, '...|[{'credit_id': '5...|11860|
|[{'cast_id': 2, '...|[{'credit_id': '5...|45325|
|[{'cast_id': 1, '...|[{'credit_id': '5...| 9091|
|[{'cast_id': 1, '...|[{'credit_id': '5...|  710|
|[{'cast_id': 1, '...|[{'credit_id': '5...| 9087|
|[{'cast_id': 9, '...|[{'credit_id': '5...|12110|
|[{'cast_id': 1, '...|[{'credit_id': '5...|21032|
|[{'cast_id': 1, '...|[{'credit_id': '5...|10858|
|[{'cast_id': 1, '...|[{'credit_id': '5...| 1408|
|[{'cast_id': 4, '...|[{'credit_id': '5...|  524|
|[{'cast_id': 6, '...|[{'credit_id': '5...| 4584|


DataFrame[cast: string, crew: string, id: int]

* transfer = udf(lambda x:eval(x), ArrayType(MapType(StringType(),StringType())))

In [19]:
# 类型转换
cres1 = cres.select('id',typetransfer('cast').alias('cast'),typetransfer('crew').alias('crew'))
cres1.dtypes

[('id', 'int'), ('cast', 'array<map<string,string>>'), ('crew', 'array<map<string,string>>')]

In [20]:
# 取出导演和主演
cres2 = cres1.select('id',getcharacter('cast').alias('cast'),getdirector('crew').alias('crew'))
cres2.show()

+-----+--------------------+--------------------+
|   id|                cast|                crew|
+-----+--------------------+--------------------+
|  862|[Tom Hanks, Tim A...|     [John Lasseter]|
| 8844|[Robin Williams, ...|      [Joe Johnston]|
|15602|[Walter Matthau, ...|     [Howard Deutch]|
|31357|[Whitney Houston,...|   [Forest Whitaker]|
|11862|[Steve Martin, Di...|     [Charles Shyer]|
|  949|[Al Pacino, Rober...|      [Michael Mann]|
|11860|[Harrison Ford, J...|    [Sydney Pollack]|
|45325|[Jonathan Taylor ...|      [Peter Hewitt]|
| 9091|[Jean-Claude Van ...|       [Peter Hyams]|
|  710|[Pierce Brosnan, ...|   [Martin Campbell]|
| 9087|[Michael Douglas,...|        [Rob Reiner]|
|12110|[Leslie Nielsen, ...|        [Mel Brooks]|
|21032|[Kevin Bacon, Bob...|       [Simon Wells]|
|10858|[Anthony Hopkins,...|      [Oliver Stone]|
| 1408|[Geena Davis, Mat...|      [Renny Harlin]|
|  524|[Robert De Niro, ...|   [Martin Scorsese]|
| 4584|[Kate Winslet, Em...|           [Ang Lee]|


In [21]:
cres2.dtypes

[('id', 'int'), ('cast', 'string'), ('crew', 'string')]

In [22]:
cres3 = cres2.select('id',rmbracket('cast').alias('Character'),rmbracket('crew').alias('Director'))
cres3.show()

+-----+--------------------+--------------------+
|   id|           Character|            Director|
+-----+--------------------+--------------------+
|  862|Tom Hanks,Tim All...|       John Lasseter|
| 8844|Robin Williams,Jo...|        Joe Johnston|
|15602|Walter Matthau,Ja...|       Howard Deutch|
|31357|Whitney Houston,A...|     Forest Whitaker|
|11862|Steve Martin,Dian...|       Charles Shyer|
|  949|Al Pacino,Robert ...|        Michael Mann|
|11860|Harrison Ford,Jul...|      Sydney Pollack|
|45325|Jonathan Taylor T...|        Peter Hewitt|
| 9091|Jean-Claude Van D...|         Peter Hyams|
|  710|Pierce Brosnan,Se...|     Martin Campbell|
| 9087|Michael Douglas,A...|          Rob Reiner|
|12110|Leslie Nielsen,Me...|          Mel Brooks|
|21032|Kevin Bacon,Bob H...|         Simon Wells|
|10858|Anthony Hopkins,J...|        Oliver Stone|
| 1408|Geena Davis,Matth...|        Renny Harlin|
|  524|Robert De Niro,Sh...|     Martin Scorsese|
| 4584|Kate Winslet,Emma...|             Ang Lee|


# 处理keywords.csv

In [23]:
kws = spark.read.csv('/Users/apple/Desktop/5003-project/moviedata/TMD/keywords.csv',multiLine=True,escape='"', header=True, inferSchema=True)

In [24]:
kws.show()

+-----+--------------------+
|   id|            keywords|
+-----+--------------------+
|  862|[{'id': 931, 'nam...|
| 8844|[{'id': 10090, 'n...|
|15602|[{'id': 1495, 'na...|
|31357|[{'id': 818, 'nam...|
|11862|[{'id': 1009, 'na...|
|  949|[{'id': 642, 'nam...|
|11860|[{'id': 90, 'name...|
|45325|                  []|
| 9091|[{'id': 949, 'nam...|
|  710|[{'id': 701, 'nam...|
| 9087|[{'id': 833, 'nam...|
|12110|[{'id': 3633, 'na...|
|21032|[{'id': 1994, 'na...|
|10858|[{'id': 840, 'nam...|
| 1408|[{'id': 911, 'nam...|
|  524|[{'id': 383, 'nam...|
| 4584|[{'id': 420, 'nam...|
|    5|[{'id': 612, 'nam...|
| 9273|[{'id': 409, 'nam...|
|11517|[{'id': 380, 'nam...|
+-----+--------------------+
only showing top 20 rows



In [25]:
# 改变类型
kws1 = kws.select('id',typetransfer('keywords').alias('keywords'))
# 提取关键词
kws2 = kws1.select('id',getcharacter('keywords').alias('keywords'))
# 删除中括号
kws3 = kws2.select('id',rmbracket('keywords').alias('keywords'))
kws3.show()
kws3

+-----+--------------------+
|   id|            keywords|
+-----+--------------------+
|  862|jealousy,toy,boy,...|
| 8844|board game,disapp...|
|15602|fishing,best frie...|
|31357|based on novel,in...|
|11862|baby,midlife cris...|
|  949|robbery,detective...|
|11860|paris,brother bro...|
|45325|                    |
| 9091|terrorist,hostage...|
|  710|cuba,falsely accu...|
| 9087|white house,usa p...|
|12110|       dracula,spoof|
|21032|wolf,dog-sledding...|
|10858|usa president,pre...|
| 1408|exotic island,tre...|
|  524|poker,drug abuse,...|
| 4584|bowling,based on ...|
|    5|hotel,new year's ...|
| 9273|africa,indigenous...|
|11517|brother brother r...|
+-----+--------------------+
only showing top 20 rows



DataFrame[id: int, keywords: string]

# 处理ratings.csv

In [26]:
df2 = spark.read.csv('/Users/apple/Desktop/5003-project/moviedata/TMD/ratings.csv',multiLine=True,escape='"', header=True, inferSchema=True)

In [27]:
#  五分制改成十分制
rats = df2.select('userId',df2['movieID'].alias('id'),(df2['rating']*2).alias('rating'))
rats.show()
rats

+------+-----+------+
|userId|   id|rating|
+------+-----+------+
|     1|  110|   2.0|
|     1|  147|   9.0|
|     1|  858|  10.0|
|     1| 1221|  10.0|
|     1| 1246|  10.0|
|     1| 1968|   8.0|
|     1| 2762|   9.0|
|     1| 2918|  10.0|
|     1| 2959|   8.0|
|     1| 4226|   8.0|
|     1| 4878|  10.0|
|     1| 5577|  10.0|
|     1|33794|   8.0|
|     1|54503|   7.0|
|     1|58559|   8.0|
|     1|59315|  10.0|
|     1|68358|  10.0|
|     1|69844|  10.0|
|     1|73017|  10.0|
|     1|81834|  10.0|
+------+-----+------+
only showing top 20 rows



DataFrame[userId: int, id: int, rating: double]

# 数据整合

bmd6
cres3
kws3
rats

In [48]:
dtpool = bmd6.join(cres3,'id').join(kws3,'id').join(rats,'id')

In [49]:
# 一个用户对应了不同的电影，主键变成双ID
dtpool.show()

+---+-------------+------------+----------+--------------------+-------------+--------------------+------+------+
| id|       genres|vote_average|vote_count|           Character|     Director|            keywords|userId|rating|
+---+-------------+------------+----------+--------------------+-------------+--------------------+------+------+
|148|Drama,Romance|         6.8|        52|Sarah Polley,Tim ...|Isabel Coixet|nurse,yugoslavia,...|   337|   8.0|
|148|Drama,Romance|         6.8|        52|Sarah Polley,Tim ...|Isabel Coixet|nurse,yugoslavia,...|   619|   4.0|
|148|Drama,Romance|         6.8|        52|Sarah Polley,Tim ...|Isabel Coixet|nurse,yugoslavia,...|  3030|   8.0|
|148|Drama,Romance|         6.8|        52|Sarah Polley,Tim ...|Isabel Coixet|nurse,yugoslavia,...|  5261|   6.0|
|148|Drama,Romance|         6.8|        52|Sarah Polley,Tim ...|Isabel Coixet|nurse,yugoslavia,...|  6697|   6.0|
|148|Drama,Romance|         6.8|        52|Sarah Polley,Tim ...|Isabel Coixet|nurse,yugo

# kws3进行处理

In [30]:
kws3.show()

+-----+--------------------+
|   id|            keywords|
+-----+--------------------+
|  862|jealousy,toy,boy,...|
| 8844|board game,disapp...|
|15602|fishing,best frie...|
|31357|based on novel,in...|
|11862|baby,midlife cris...|
|  949|robbery,detective...|
|11860|paris,brother bro...|
|45325|                    |
| 9091|terrorist,hostage...|
|  710|cuba,falsely accu...|
| 9087|white house,usa p...|
|12110|       dracula,spoof|
|21032|wolf,dog-sledding...|
|10858|usa president,pre...|
| 1408|exotic island,tre...|
|  524|poker,drug abuse,...|
| 4584|bowling,based on ...|
|    5|hotel,new year's ...|
| 9273|africa,indigenous...|
|11517|brother brother r...|
+-----+--------------------+
only showing top 20 rows



In [31]:
df1 = kws3.join(bmd5,'id').select('id','keywords','title','tagline')
df1 = df1.dropna(how='any')
df1.show()
df1
df2 = df1.select('id','title','keywords',cutwords('tagline').alias('tagline'))
df2

+-----+--------------------+--------------------+--------------------+
|   id|            keywords|               title|             tagline|
+-----+--------------------+--------------------+--------------------+
| 8844|board game,disapp...|             Jumanji|Roll the dice and...|
|15602|fishing,best frie...|    Grumpier Old Men|Still Yelling. St...|
|31357|based on novel,in...|   Waiting to Exhale|Friends are the p...|
|11862|baby,midlife cris...|Father of the Bri...|Just When His Wor...|
|  949|robbery,detective...|                Heat|A Los Angeles Cri...|
|11860|paris,brother bro...|             Sabrina|You are cordially...|
|45325|                    |        Tom and Huck|The Original Bad ...|
| 9091|terrorist,hostage...|        Sudden Death|Terror goes into ...|
|  710|cuba,falsely accu...|           GoldenEye|No limits. No fea...|
| 9087|white house,usa p...|The American Pres...|Why can't the mos...|
|21032|wolf,dog-sledding...|               Balto|Part Dog. Part Wo...|
|10858

DataFrame[id: int, title: string, keywords: string, tagline: array<string>]

In [ ]:

# 这里应该在做一个语义分析，TfidfVectorizer是tflearn中的包。
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

# 这个方法的解释在上方，fit_transform生成一个矩阵
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])
count.get_feature_names()

In [159]:
# SAMPLE
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")
], ["label", "sentence"])
sentenceData.show()
sentenceData

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi I heard about ...|
|  0.0|I wish Java could...|
|  1.0|Logistic regressi...|
+-----+--------------------+



DataFrame[label: double, sentence: string]

In [160]:
# SAMPLE
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsData = tokenizer.transform(sentenceData)
wordsData.show()
wordsData

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|  0.0|Hi I heard about ...|[hi, i, heard, ab...|
|  0.0|I wish Java could...|[i, wish, java, c...|
|  1.0|Logistic regressi...|[logistic, regres...|
+-----+--------------------+--------------------+



DataFrame[label: double, sentence: string, words: array<string>]

In [161]:
# SAMPLE
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)
# alternatively, CountVectorizer can also be used to get term frequency vectors

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

rescaledData.select("label", "features").show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(20,[0,5,9,17],[0...|
|  0.0|(20,[2,7,9,13,15]...|
|  1.0|(20,[4,6,13,15,18...|
+-----+--------------------+



In [171]:
tokenizer = Tokenizer(inputCol="tagline", outputCol="words")
# ,"keywords"
wordsData = tokenizer.transform(df1)
# 之前这里报错，是因为前面没有drop空白行
wordsData.select('id','tagline','words').show()
wordsData

+-----+--------------------+--------------------+
|   id|             tagline|               words|
+-----+--------------------+--------------------+
| 8844|Roll the dice and...|[roll, the, dice,...|
|15602|Still Yelling. St...|[still, yelling.,...|
|31357|Friends are the p...|[friends, are, th...|
|11862|Just When His Wor...|[just, when, his,...|
|  949|A Los Angeles Cri...|[a, los, angeles,...|
|11860|You are cordially...|[you, are, cordia...|
|45325|The Original Bad ...|[the, original, b...|
| 9091|Terror goes into ...|[terror, goes, in...|
|  710|No limits. No fea...|[no, limits., no,...|
| 9087|Why can't the mos...|[why, can't, the,...|
|21032|Part Dog. Part Wo...|[part, dog., part...|
|10858|Triumphant in Vic...|[triumphant, in, ...|
| 1408|The Course Has Be...|[the, course, has...|
|  524|No one stays at t...|[no, one, stays, ...|
| 4584|Lose your heart a...|[lose, your, hear...|
|    5|Twelve outrageous...|[twelve, outrageo...|
| 9273|New animals. New ...|[new, animals., n...|


DataFrame[id: int, keywords: string, title: string, tagline: string, words: array<string>]

In [173]:
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)
# alternatively, CountVectorizer can also be used to get term frequency vectors

In [177]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)
rescaledData.select("id","features").show(truncate=False)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id   |features                                                                                                                                                                                                                                                                                                       |
+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|8844 |(20,[3,4,10,13,14,15],[0.9417978315184856,1.3017304533497

In [201]:
result = rescaledData.select("id","features")
type(result)
result.show()

+-----+--------------------+
|   id|            features|
+-----+--------------------+
| 8844|(20,[3,4,10,13,14...|
|15602|(20,[0,4,8,10,16,...|
|31357|(20,[2,4,5,6,10,1...|
|11862|(20,[0,1,3,4,5,6,...|
|  949|(20,[6,7,9,10,12]...|
|11860|(20,[3,5,7,8,10,1...|
|45325|(20,[6,10,14,18],...|
| 9091|(20,[0,5,11,15],[...|
|  710|(20,[6,11,15,16],...|
| 9087|(20,[0,2,4,5,8,9,...|
|21032|(20,[0,2,5,8,10],...|
|10858|(20,[0,3,5,7,10,1...|
| 1408|(20,[0,1,2,3,6,7,...|
|  524|(20,[1,4,6,8,10,1...|
| 4584|(20,[0,2,3,4,8,10...|
|    5|(20,[0,2,3,4,5,7,...|
| 9273|(20,[5,9,13,16,17...|
|11517|(20,[0,7,10,14,19...|
| 8012|(20,[0,1,3,4,10,1...|
| 1710|(20,[1,3,4,5,6,7,...|
+-----+--------------------+
only showing top 20 rows



In [188]:
from pyspark.mllib.linalg.distributed import *
rows = sc.parallelize([[1, 2], [1, 5]])
mat = RowMatrix(rows)
type(mat)

<class 'pyspark.mllib.linalg.distributed.RowMatrix'>

In [32]:
df2

DataFrame[id: int, title: string, keywords: string, tagline: array<string>]

In [44]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
hashingTF = HashingTF(inputCol="tagline", outputCol="rawFeatures", numFeatures=1)
featurizedData = hashingTF.transform(df2)
idf = IDF(inputCol="rawFeatures", outputCol="features").fit(featurizedData)
tfidf = idf.transform(featurizedData)

In [45]:
from pyspark.ml.feature import Normalizer
normalizer = Normalizer(inputCol="features", outputCol="norm")
data = normalizer.transform(tfidf)

In [46]:
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix
mat = IndexedRowMatrix(
    data.select("ID", "norm")\
        .rdd.map(lambda row: IndexedRow(row.ID, row.norm.toArray()))).toBlockMatrix()
type(mat)
dot = mat.multiply(mat.transpose())
# dot.toLocalMatrix().toArray()
type(dot)

<class 'pyspark.mllib.linalg.distributed.BlockMatrix'>

In [47]:
dot.toLocalMatrix().toArray()

Name: org.apache.toree.interpreter.broker.BrokerException
Message: Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.2.1/libexec/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/usr/local/Cellar/apache-spark/2.2.1/libexec/python/lib/py4j-0.10.4-src.zip/py4j/protocol.py", line 319, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: An error occurred while calling o589.toLocalMatrix.
: java.lang.IllegalArgumentException: requirement failed: The length of the values array must be less than Int.MaxValue. Currently numRows * numCols: 216266852025
	at scala.Predef$.require(Predef.scala:224)
	at org.apache.spark.mllib.linalg.distributed.BlockMatrix.toLocalMatrix(BlockMatrix.scala:307)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAcce

In [36]:
print(dot)

In [37]:
print(mat)

In [ ]:
import pyspark.sql.functions as psf
dot_udf = psf.udf(lambda x,y: float(x.dot(y)), DoubleType())
data.alias("i").join(data.alias("j"), psf.col("i.ID") < psf.col("j.ID"))\
    .select(
        psf.col("i.ID").alias("i"), 
        psf.col("j.ID").alias("j"), 
        dot_udf("i.norm", "j.norm").alias("dot"))\
    .sort("i", "j")\
    .show()